In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np

# !nbdev_build_lib

In [3]:
categories = pd.read_sql("select distinct(category) from food.foods",engine)

In [10]:
categories.to_sql('food_categories',engine,schema = 'food',if_exists = 'append',index = False)

In [6]:
pd.read_sql("select * from food.food_categories",engine)

,id,category,clip
0,11,Pears,"[0.064467475, 0.007819213, 0.0068222187, 0.002..."
1,1,Broccoli,"[0.02630453, 0.008566782, 0.047704235, 0.00536..."
2,2,Nachos,"[-0.00556623, -0.02693382, 0.011428544, -0.024..."
3,3,Soy-based condiments,"[0.008842907, 0.026064698, -0.0032581512, -0.0..."
4,4,Cheese sandwiches (single code),"[0.03734121, 0.022437142, 0.03321014, -0.02276..."
...,...,...,...
162,163,Baby food: meat and dinners,"[-0.008153972, -0.014811312, 0.015918016, 0.00..."
163,164,Other Mexican mixed dishes,"[0.025341043, -0.021919917, 0.035512406, -0.00..."
164,165,"Flavored milk, whole","[-0.0031074989, 0.0091486145, -0.008182965, 0...."
165,166,"Formula, prepared from powder","[0.0060971905, 0.038108524, 0.064308636, 0.016..."


In [3]:
table = 'food_categories'

In [4]:
import os
os.environ["HF_DATASETS_OFFLINE"] = "2"
os.environ["TRANSFORMERS_OFFLINE"] = "2"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from food.tools import *
from food.psql import *
from food.paths import *
from PIL import Image
import custom_pandas as cpd
from tqdm import tqdm
from sqlalchemy import update

query = f"""select id,category
             from food.{table}
            where clip is Null """

total = engine.execute(f'select count(*) from ({query}) a').one()

bs = 1
pd_iter = cpd.read_sql_query(query, engine, chunksize=bs, index_col='id')

import requests


In [5]:

for inp in tqdm(pd_iter, desc="clip food inference", total=total[0] // bs):  
    text = inp['category'].iloc[0]
    clip = requests.post(f'https://guru.skynet.center/text2vector/?text={text}').json()
    stmt = (update(Food_categories).
            values(clip=clip).
            where(Food_categories.id == int(inp.index[0])))
    engine.execute(stmt)

clip food inference: 100%|██████████| 167/167 [00:07<00:00, 22.21it/s]
